# Xử lí dữ liệu Covid19 theo ngày sử dụng Spark, AWS EMR
## Load dữ liệu ngày cần phân tích lên hệ thống Hadoop - AWS EMR
1. Chọn ngày cần xử lí

In [ ]:
from datetime import date

day = date(2022,11,10)

2. Định nghĩa nguồn dữ liệu và nơi lưu trữ file phân tán (AWS S3)

In [ ]:
S3_DATA_SOURCE_PATH = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+ day.strftime('%m-%d-%Y')+'.csv'
S3_DATA_OUTPUT_PATH = 's3://nhom5-cloud/covid19/' + day.strftime('%d-%m-%Y')

3. Định nghĩa cấu trúc của dữ liệu

In [ ]:
from pyspark.sql.types import *
schema = StructType([ 
    StructField("FIPS", StringType(), True),
    StructField("Admin2", StringType(), True),
    StructField("Province_State", StringType(), True),
    StructField("Country_Region", StringType(), True),
    StructField("Last_Update", StringType(), True),
    StructField("Lat", StringType(), True),
    StructField("Long_", StringType(), True),
    StructField("Confirmed", StringType(), True),
    StructField("Deaths", StringType(), True),
    StructField("Recovered", StringType(), True),
    StructField("Active", StringType(), True),
    StructField("Combined_Key", StringType(), True),
    StructField("Incident_Rate", StringType(), True),
    StructField("Case_Fatality_Ratio", StringType(), True),
 ])

4. Tạo phiên viết dữ liệu

In [ ]:
from pyspark.sql import SparkSession

spark_write = SparkSession.builder.appName('Nhom5-Cloud-Write').getOrCreate()

5. Đọc dữ liệu và định nghĩa dữ liệu theo cấu trúc

In [ ]:
import pandas as pd

df = pd.read_csv(S3_DATA_SOURCE_PATH).fillna('')
all_data = spark_write.createDataFrame(df,schema=schema)

6. Lưu dữ liệu dưới dạng file phân tán lên S3

In [ ]:
# ghi dữ liệu nếu chưa tồn tại
all_data.write.mode('ignore').parquet(S3_DATA_OUTPUT_PATH)

## Lấy dữ liệu theo ngày và xử lí
1. Tạo phiên đọc dữ liệu và đọc dữ liệu từ file phân tán

In [ ]:
spark_read = SparkSession.builder.appName('Nhom5-Cloud-Read').getOrCreate()
df_read = spark_read.read.parquet(S3_DATA_OUTPUT_PATH)

2. In ra một số dữ liệu đọc được, số dòng, số cột của tập dữ liệu 

In [ ]:
df_read.show(truncate=False)

rows = df_read.count()
print(f"Data from {day} have {rows} rows.")

cols = len(df_read.columns)
print(f"Data from {day} have {cols} columns.")

3. Xử lí dữ liệu
* Top 10 đất nước có số lượng mắc covid nhiều nhất

In [ ]:
from pyspark.sql import functions as func

# Chuyển kiểu dữ liệu của cột 'Confirmed' kiểu String sang Int
df_read.withColumn('Confirmed', df_read['Confirmed'].cast(IntegerType()))
# Chuyển các dòng dữ liệu bị null thành 0
df_read.fillna(value=0,subset=['Confirmed'])
# Tính tổng dữ liệu cột 'Confirmed' theo 'Country_Region'
df_read.select('Country_Region','Confirmed').groupBy('Country_Region').agg(func.sum('Confirmed')).sort(df_read['Confirmed'].desc()).show(10)

* Top 10 đất nước có số lượng tử vong vì covid nhiều nhất

In [ ]:
# Chuyển kiểu dữ liệu của cột 'Deaths' kiểu String sang Int
df_read.withColumn('Deaths', df_read['Deaths'].cast(IntegerType()))
# Chuyển các dòng dữ liệu bị null thành 0
df_read.fillna(value=0,subset=['Deaths'])
# Tính tổng dữ liệu cột 'Deaths' theo 'Country_Region'
df_read.select('Country_Region','Deaths').groupBy('Country_Region').agg(func.sum('Deaths')).sort(df_read['Deaths'].desc()).show(10)

* Top 10 đất nước có số lượng hồi phục sau mắc covid nhiều nhất

In [ ]:
# Chuyển kiểu dữ liệu của cột 'Recovered' kiểu String sang Int
df_read.withColumn('Recovered', df_read['Recovered'].cast(IntegerType()))
# Chuyển các dòng dữ liệu bị null thành 0
df_read.fillna(value=0,subset=['Recovered'])
# Tính tổng dữ liệu cột 'Recovered' theo 'Country_Region'
df_read.select('Country_Region','Recovered').groupBy('Country_Region').agg(func.sum('Recovered')).sort(df_read['Recovered'].desc()).show(10)